In [1]:
import torch
torch.cuda.is_available()

True

In [2]:

#from preprocessing import tokenize, export_to_csv
from gsdmm import MovieGroupProcess
import pickle
import pandas as pd
import numpy as np
%matplotlib inline
import time

In [3]:
!pip install pytorch-pretrained-bert
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from IPython.display import clear_output
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=False)
import re

In [4]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

import spacy

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/skarra7/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def stopword(text):
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def stemwords(text):    
    ps = PorterStemmer()
    text = text.lower().split()
    stemmed_words = [ps.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

In [6]:
!pip install -U pip setuptools wheel

In [7]:
#This pickle file is obatined by filtering english language users who have atleast 20 tweets on their home timeline
#It is straightforward hence we generate a pickle file of filtered dataset and use here directly.

file = open("filtered_english.pkl",'rb')
df = pickle.load(file)
file.close()

In [8]:
nlp = spacy.load('en_core_web_sm',disable = ['parser','entity'])
import time
start_time = time.time()

file = open("filtered_english.pkl",'rb')
df = pickle.load(file)
file.close()

df.drop(["engaging_user_id","tweet_id","tweet_timestamp","reply_timestamp","retweet_timestamp","retweet_with_comment_timestamp","like_timestamp"], axis = 1)
df['text_tokens']= df['text_tokens'].str.split('\t').apply(lambda x:list(map(int, x)))
print ("--- %s seconds ---" % (time.time() - start_time))  

df['text_tokens'] = df['text_tokens'].apply(lambda x:tokenizer.convert_ids_to_tokens(x))
print ("--- %s seconds ---" % (time.time() - start_time))  

--- 50.20185446739197 seconds ---
--- 72.20108079910278 seconds ---


In [9]:

def conv(s):
    l = []
    for i in range(len(s)):
        if s[i][:2]!='##':
            l.append(s[i])
        else:
            l[-1]=l[-1]+s[i][2:]
    return l


df['text_tokens'] = df['text_tokens'].apply(lambda x: conv(x))

In [10]:
df['text_tokens'].head()

57    [[CLS], Even, by, the, standards, of, Trump, a...
58    [[CLS], EXCLUSIVE, :, Manuscript, of, Bolton, ...
61    [[CLS], Whoever, the, patriot, is, that, leake...
62    [[CLS], If, anything, the, president, does, to...
70    [[CLS], Flood, my, guts, ., ., #, bigdick, #, ...
Name: text_tokens, dtype: object

In [11]:
df['text_tokens']= df['text_tokens'].apply(lambda x: " ".join([y for y in x]))
print ("--- %s seconds ---" % (time.time() - start_time))  

for r in (("[CLS]", ""),("[SEP]",""),("[UNK]",""),("RT","")):
    df['text_tokens'] = df['text_tokens'].apply(lambda x: x.replace(*r))
print ("--- %s seconds ---" % (time.time() - start_time))  

df["text_tokens"] = df["text_tokens"].apply(lambda x:' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split()))
print ("--- %s seconds ---" % (time.time() - start_time))  

df["text_tokens"] = df["text_tokens"].apply(lambda x: stemwords(x))
print ("--- %s seconds ---" % (time.time() - start_time))

df["text_tokens"] = df["text_tokens"].apply(lambda x: stopword(x))
print ("--- %s seconds ---" % (time.time() - start_time))  


#df.to_pickle('text_for_topics_1.pkl')
 


--- 106.4303925037384 seconds ---
--- 110.95485138893127 seconds ---
--- 152.6514790058136 seconds ---
--- 803.4099097251892 seconds ---
--- 1039.5841271877289 seconds ---


In [12]:
df['text_tokens'].head()

57    even standard trump apologist believ anyth exc...
58    exclus manuscript bolton book say told ag barr...
61         whoever patriot leak bolton manuscript thank
62    anyth presid doe get elect ok could take step ...
70    flood gut bigdick breed bareback raw repost ju...
Name: text_tokens, dtype: object

In [13]:
df['text_tokens'] = df['text_tokens'].apply(lambda x: x.split())

In [14]:
docs = df['text_tokens'].tolist()
vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)

print("Voc size:", n_terms)
print("Number of documents:", len(docs))

Voc size: 1361455
Number of documents: 3409013


In [15]:
mgp = MovieGroupProcess(K=100, alpha=0.1, beta=0.1, n_iters=20)

In [ ]:
# Train a new model 

# Init of the Gibbs Sampling Dirichlet Mixture Model algorithm

start_time = time.time()
vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)
n_docs = len(docs)

# Fit the model on the data given the chosen seeds
y = mgp.fit(docs, n_terms)
print ("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topics :', sorted(doc_count))
print('*'*20)

# Topics sorted by document inside
top_index = doc_count.argsort()[-10:][::-1]
print('Most important clusters (by number of docs inside):', top_index)
print('*'*20)

In [ ]:
def top_words(distribution, top_index, num_words):
    for topic in top_index:
        pairs = sorted([(k, v) for k, v in distribution[topic].items()], key=lambda x: x[1], reverse=True)
        a = pairs[:num_words]
        c = [i[0] for i in a]
        print (c)
        print('.'*30)

b = top_words(mgp.cluster_word_distribution, top_index, 1000)        

In [ ]:
df['text_tokens'] = df['text_tokens'].apply(lambda x: mgp.choose_best_label(x)[0])

In [ ]:
df = df.reset_index(drop = True)

In [ ]:
with open('gsdmm_topics.pkl', "wb") as f:
     pickle.dump(df, f)
     f.close()